In [41]:

from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline
from pyspark.sql import functions as fn, Row
from pyspark import sql
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression

import matplotlib.pyplot as plt
import pandas as pd

In [44]:
train = pd.read_csv('sampleTrain.csv', sep=',')


In [45]:
train.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,...,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000,44470.000000
mean,0.233011,1.097976,128.950505,0.642905,0.221723,1.353564,47.892962,506.026265,0.905060,0.535597,...,0.164358,605.370772,0.003620,4.632184,0.024264,0.007870,1147.982816,3.636272,608.822734,0.471774
std,0.580972,1.705380,167.193098,1.122161,0.583680,2.637361,27.387283,626.436077,1.524653,0.704919,...,0.480937,1498.961954,0.069439,30.394854,0.169448,0.092592,1175.015832,2.397967,739.922463,0.307350
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,151.700000,2.000000,0.000000,0.195900
50%,0.000000,0.000000,83.270000,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,682.800000,3.000000,0.000000,0.458300
75%,0.000000,2.000000,183.800000,1.000000,0.000000,2.000000,71.000000,1170.000000,1.000000,1.000000,...,0.000000,0.017150,0.000000,0.000000,0.000000,0.000000,1956.000000,5.000000,1495.000000,0.740700
max,8.000000,16.000000,2935.000000,17.000000,12.000000,43.000000,100.000000,2122.000000,29.000000,6.000000,...,13.000000,14600.000000,3.000000,1142.000000,5.000000,3.000000,9124.000000,44.000000,1958.000000,1.000000


In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44470 entries, 0 to 44469
Data columns (total 26 columns):
assists            44470 non-null int64
boosts             44470 non-null int64
damageDealt        44470 non-null float64
DBNOs              44470 non-null int64
headshotKills      44470 non-null int64
heals              44470 non-null int64
killPlace          44470 non-null int64
killPoints         44470 non-null int64
kills              44470 non-null int64
killStreaks        44470 non-null int64
longestKill        44470 non-null float64
matchDuration      44470 non-null int64
matchType          44470 non-null object
maxPlace           44470 non-null int64
numGroups          44470 non-null int64
rankPoints         44470 non-null int64
revives            44470 non-null int64
rideDistance       44470 non-null float64
roadKills          44470 non-null int64
swimDistance       44470 non-null float64
teamKills          44470 non-null int64
vehicleDestroys    44470 non-null int64
wa

In [47]:
trian=train.dropna()

In [49]:
train['matchType'].unique()

array(['squad', 'squad-fpp', 'duo-fpp', 'solo-fpp', 'duo', 'solo',
       'normal-squad-fpp', 'normal-squad', 'crashfpp', 'normal-duo-fpp',
       'flarefpp', 'flaretpp', 'normal-solo-fpp', 'normal-solo',
       'crashtpp'], dtype=object)

In [55]:
# celan match type data ( there are too many different types )
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'
train['matchType'] = train['matchType'].map(mapper)
train['winPlacePerc'] = train['winPlacePerc'].astype('float')

In [62]:
# create pyspark dataframe
train_df = spark.createDataFrame(train)

In [63]:
# transfer categorical data of match type to numerical data
train_df = train_df.\
                    withColumn('solo',(fn.col('matchType') == 'solo').cast('int')).\
                    withColumn('duo',(fn.col('matchType') == 'duo').cast('int')).\
                    withColumn('squad',(fn.col('matchType') == 'squad').cast('int'))
train_df = train_df.drop('matchType')

In [70]:
train_df.show(5)

+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------------+-------------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+-------------------+----+---+-----+
|assists|boosts|damageDealt|DBNOs|headshotKills|heals|killPlace|killPoints|kills|killStreaks|      longestKill|matchDuration|maxPlace|numGroups|rankPoints|revives|rideDistance|roadKills|swimDistance|teamKills|vehicleDestroys|walkDistance|weaponsAcquired|winPoints|       winPlacePerc|solo|duo|squad|
+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------------+-------------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+-------------------+----+---+-----+
|      1|     1|      167.5|    1|            0|    5|        8|         0|    3|          2|       

In [96]:
train_df = train_df.withColumnRenamed('winPlacePerc', 'label')

In [97]:
#input cols
inputcols = train_df.columns
inputcols.remove('label')

## Vector Assemble
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(inputcols).setOutputCol('features')

In [98]:
#split data
(trainingData, testingData) = train_df.randomSplit([0.7, 0.3])

In [99]:
# create pipeline
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol = 'features')

# Chain indexer and forest in a Pipeline
rf_pipeline = Pipeline(stages=[va, rf])

# Train model.  This also runs the indexer.
model = rf_pipeline.fit(trainingData)

In [101]:
# Make predictions.
predictions = model.transform(testingData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+-------------------+------+--------------------+
|         prediction| label|            features|
+-------------------+------+--------------------+
| 0.3195315646547384|   0.0|(27,[6,11,12,13,1...|
|0.46642179077806345|0.2703|(27,[6,7,11,12,13...|
| 0.6033055143611297|0.5556|(27,[6,11,12,13,1...|
| 0.3589401377210678|   0.0|(27,[6,7,11,12,13...|
| 0.5825586441674131|0.9149|(27,[6,11,12,13,1...|
+-------------------+------+--------------------+
only showing top 5 rows



In [102]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 0.122188


In [103]:
rfModel = model.stages[1]
print(rfModel)  # summary only

RandomForestRegressionModel (uid=RandomForestRegressor_40388cc5d58b3ade017e) with 20 trees
